In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
import matplotlib.transforms as transforms
import matplotlib
import pandas as pd
import os
import os.path
from scipy.optimize import curve_fit

In [5]:
dname = os.path.dirname(os.getcwd()) + '/data_loading/data_v2_quotes/'
dname

'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts/data_loading/data_v2_quotes/'

In [6]:
ACCURACY = np.load(dname + 'ACCURACY_New.npy')
acc_first_prop = np.load(dname + 'Acc_first_prop_New.npy')
acc_second_prop = np.load(dname + 'acc_second_prop_New.npy')
acc_third_prop = np.load(dname + 'acc_third_prop_New.npy')

In [7]:
n_sujs, n_blocs, n_trials = ACCURACY.shape
# ACCURACY.shape
acc_first_prop.shape
acc_second_prop.shape
# ACCURACY[0,:,:] 

(49, 8, 16)

In [10]:
print(acc_first_prop[0,0,:])
acc_first_prop_reshaped = np.reshape(acc_first_prop,(49,128))
acc_second_prop_reshaped = np.reshape(acc_second_prop,(49,128))
acc_third_prop_reshaped = np.reshape(acc_third_prop,(49,128))

[1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1.]
[1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1.]


In [ ]:
def plot_accuracy_indi(subj_means, fig_name, fig_title, chance_level=False):
    # Define figure
    fig, ax = plt.subplots(1, 1, figsize=(9, 4), dpi=300, facecolor='w')

    # Individual data
    for i in range(subj_means.shape[0]-40):
        ax.plot(
            np.arange(n_blocs),
            subj_means[i,:]
        )
    if chance_level:
        ax.axhline(y=chance_level, color='r', linestyle='--' )
    # Aesthetics
    ax.set_title(fig_title, fontweight='bold')
    ax.set_ylim(0.3, 1.05)
    ax.set_xlabel('Block #')
    ax.set_ylabel('Accuracy')

    # Hide the right and top spines
    ax.spines.right.set_visible(False)
    ax.spines.top.set_visible(False)

    plt.tight_layout()
    # fig.savefig(fig_name)

In [ ]:
def sigmoid(x, x0, k):
    y = 1 / (1 + np.exp(-k*(x-x0)))
    return y

def linear(x, w, b):
    y = x * w + b
    return y

def relu(x, w, b1, b2):
    y = np.maximum((x * w - b1),0) + b2
    return y

def heaviside(x, b1, b2):
    y = 1 * (x + b1 > 0) * b2
    return y

In [ ]:
def part_fit(function, data, n_params, p0, bound):
    """make fit to each participant and return the params

    Args:
        function (_type_): _description_
        data (_type_): _description_
    """
    all_popt = np.full((data.shape[0], n_params), np.nan)
    # loop over each participant
    x_data = np.linspace(1, data.shape[1], 8)

    for i in range(data.shape[0]):
        y_data = data[i,:]

        popt, pcov = curve_fit(function, x_data, y_data, p0, method='dogbox',bounds=bound, maxfev=5000)
        
        all_popt[i,:] = popt

    return all_popt

In [ ]:
p0 = [max(y_data), 1] # this is an mandatory initial guess
bound = ([1, -np.inf], [np.inf, np.inf])

all_popt = part_fit(sigmoid, subj_acc_second, 2, p0, bound)
df = pd.DataFrame(all_popt, columns = ['x0','k'])
df.to_csv("params/sigmoid.csv")
print(all_popt.shape)